# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
# File to Load (Remember to Change These)
city_data_load = "WeatherPy/WeatherPy.csv"
#student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
city_df = pd.read_csv(city_data_load)
#student_data = pd.read_csv(student_data_to_load)
city_df.dropna(inplace = True) 
city_df

City Country  Max Temp        Date  Cloudiness  \
0                Punta Arenas      CL     48.20  1619396494          75   
1                       Cervo      ES     55.00  1619396261         100   
2                    Karratha      AU     78.89  1619396262           0   
3                      Eureka      US     55.00  1619396495          93   
4              Port Elizabeth      ZA     59.00  1619396570           0   
..                        ...     ...       ...         ...         ...   
595                   Gigmoto      PH     78.48  1619396888          99   
596                    Baiyin      CN     39.20  1619396889          75   
597                 Kununurra      AU     82.40  1619396299          98   
598  Arrondissement de Thiers      FR     55.40  1619396889           0   
599                 Maniitsoq      GL     40.60  1619396567         100   

         Lat       Lng  Wind Speed  Humidity  
0   -53.1500  -70.9167       10.36        87  
1    43.6702   -7.4101        1.01        88  
2   -20.7377  116.8463        9.10        42  
3    40.8021 -124.1637        3.00        77  
4   -33.9180   25.5701        8.05        88  
..       ...       ...         ...       ...  
595  13.7796  124.3900        7.09        75  
596  36.5583  104.2081        2.24        70  
597 -15.7667  128.7333       12.66        36  
598  45.8333    3.5000        4.61        47  
599  65.4167  -52.9000        4.68        77  

[596 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

In [17]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
city1 = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70)]
city2 = city1.loc[(city1["Wind Speed"] < 10)]
city3 = city2.loc[city2["Cloudiness"] == 0]
city3

City Country  Max Temp        Date  Cloudiness      Lat  \
2                 Karratha      AU     78.89  1619396262           0 -20.7377   
58   Conceição do Araguaia      BR     73.40  1619396736           0  -8.2578   
60     Passo de Camaragibe      BR     75.20  1619396737           0  -9.2368   
83             Canavieiras      BR     76.19  1619396743           0 -15.6750   
95                   Dondo      MZ     71.60  1619396746           0 -19.6094   
186                 Kibala      TD     77.00  1619396772           0   9.1104   
260                 Riyadh      SA     78.80  1619396792           0  24.6877   
424             Mossamedes      AO     76.57  1619396644           0 -15.1961   
452                 Maceió      BR     75.20  1619396627           0  -9.6658   
569             Esmeraldas      BR     75.20  1619396881           0 -19.7625   
588                  Patti      IN     71.60  1619396886           0  31.2808   
590                Estelle      US     75.99  1619396887           0  29.8458   

          Lng  Wind Speed  Humidity  
2    116.8463        9.10        42  
58   -49.2647        2.30        94  
60   -35.4901        3.44        88  
83   -38.9472        6.64        82  
95    34.7431        3.51        94  
186   18.3463        5.30        73  
260   46.7219        3.44        20  
424   12.1522        9.75        67  
452  -35.7353        3.44        88  
569  -44.3139        2.30        64  
588   74.8572        2.35        68  
590  -90.1067        2.95        57

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = pd.DataFrame({})
hotel_df


hotel_df["Hotel Name"]= ""
hotel_df["City"] = ""
hotel_df["Country"]=""


hotel_df

Empty DataFrame
Columns: [City, Country, Hotel Name]
Index: []

In [ ]:
# geocoordinates
target_coordinates = city3["Lat"],city3["Lng"]
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [20]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

NameError: name 'response' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
